# Item 15: Be Cautious When Relying on `dict` Insertion Ordering

In Python 3.5 and before, iterating over a dict would return keys in arbitrary order. The order of iteration would not match the order un which the items were inserted.

In [1]:
# Starting with Python 3.6 ditctionaries, will preserve insertion ordering
baby_names = {
    'cat': 'kitten',
    'dog': 'puppy'
}
print(baby_names)

{'cat': 'kitten', 'dog': 'puppy'}


## Python 3.5 code has been omitted from these item due to kernel issues

In [3]:
class MyClass:
    def __init__(self):
        self.alligator = 'hatchling'
        self.elephant = 'calf'

a = MyClass()
for key, value in a.__dict__.items():
    print(f'{key} = {value}')

alligator = hatchling
elephant = calf


The way that Python dictionaries preserve insetion is now part of the Pythonlanguage specification. For the language features above, we can rely on this behavior and even make it part of the APIs we design for our classes and functions.

However, we shouldn't always assume that insertion ordering behavior will be present when we're handling dictionaries. Python makes it easy for programmers to define our own custom container types that emulate the standard *protocols* matching `list`, `dict`, and other types. Python is not statically types, so most code relies on *duck-typing*-where an object's behavior is ts de facto type-instead of rigid class hierarchies. **This can result in surprising gotchas.**

In [4]:
# Program to show results of a content fot the cutest baby animal
votes = {
    'otter': 1281,
    'polar bear': 587,
    'fox': 863
}

In [9]:
# Function to process voting data
def populate_ranks(votes, ranks):
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True)
    for i, name in enumerate(names, 1):
        ranks[name] = i

In [10]:
# Function to tell which animal won contest
def get_winner(ranks):
    return next(iter(ranks))

In [11]:
# Prove that both functions work as intended
ranks = {}
populate_ranks(votes, ranks)
print(ranks)
winner = get_winner(ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar bear': 3}
otter


In [15]:
# Reuslts should now be displayed in alphabetical order
# To accomplish this, we can use the collections.abc built-int module to define a new dictionary-like class
# hat iterates its content in alphabetical order
from collections.abc import MutableMapping

class SortedDict(MutableMapping):
    def __init__(self):
        self.data = {}

    def __getitem__(self, key):
        return self.data[key]

    def __setitem__(self, key, value):
        self.data[key] = value

    def __delitem__(self, key):
        del self.data[key]

    def __iter__(self):
        keys = list(self.data.keys())
        keys.sort()
        for key in keys:
            yield key
        
    def __len__(self):
        return len(self.data)

In [16]:
# We can ue a SortedDict instance in place of a standard dict with the functions from before and no errors will
# raised since this class conforms to the protocol of a standard dictionary. However, the result is incorrect
sorted_ranks = SortedDict()
populate_ranks(votes, sorted_ranks)
print(sorted_ranks.data)
winner = get_winner(sorted_ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar bear': 3}
otter


The problem with the previous chunk of code is that the implementation of `get_winner` assumes that the dictionary's iteration is in insertion order to match `populate_ranks`. This code is using `SortedDict` instead of `dict`, so that assumption is no longer true. Thus, the value returned from the winner is `'fox'` which is alphabetically first.

We can mitigate this problem in three ways:

In [17]:
# First, we can reimplement the get_winner function to no longer assume that the ranks dictionary has a specific
# iteration order. This is the most conservative and robust solution
def get_winner(ranks):
    for name, rank in ranks.items():
        if rank == 1:
            return name

winner = get_winner(sorted_ranks)
print(winner)

otter


In [18]:
# The second approach is to add an explicit check to the top of the function to ensure that the type of ranks
# matches our expectations, and to raise an exception if not. This solution likely has better runtime performance
# than the more conservative approach above
def get_winner(ranks):
    if not isinstance(ranks, dict):
        raise TypeError('Must provide a dict instance')
    return next(iter(ranks))

get_winner(sorted_ranks)


TypeError: Must provide a dict instance

In [20]:
# The third alternative is to use type annotations to enforce that the value passed to get_winner is a dict 
# instance and not a MutableMapping with dictionary-like behavior 
# TODO: run mypy tool in strict mode
# This correctly detects the mismatch between the dict and MutableMapping types and flags the incorrect usae as
# an error. This solution provides the best mix of static type safety and runtime performance.
from typing import Dict, MutableMapping

def populate_ranks(votes: Dict[str, int],
                   ranks: Dict[str, int]) -> None:
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True)
    for i, name in enumerate(names, 1):
        ranks[name] = i

def get_winner(ranks: Dict[str, int]) -> str:
    return next(iter(ranks))

class SortedDict(MutableMapping[str, int]):
    def __init__(self):
        self.data = {}

    def __getitem__(self, key):
        return self.data[key]

    def __setitem__(self, key, value):
        self.data[key] = value

    def __delitem__(self, key):
        del self.data[key]

    def __iter__(self):
        keys = list(self.data.keys())
        keys.sort()
        for key in keys:
            yield key
        
    def __len__(self):
        return len(self.data)

votes = {
    'otter': 1281,
    'polar bear': 587,
    'fox': 863
}

sorted_ranks = SortedDict()
populate_ranks(votes, sorted_ranks)
print(sorted_ranks.data)
winner = get_winner(sorted_ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar bear': 3}
fox
